In [1]:
import pandas as pd
import numpy as np
%config InlineBackend.figure_format = 'retina'
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
import scipy
from matplotlib.colors import ListedColormap
from functools import partial
from sklearn.model_selection import KFold
import numpy as np
import scipy.io as sio
import numpy as np
import seaborn as sns
from sklearn import tree
from sklearn import datasets
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

In [2]:
df = pd.read_csv('wine.data', header=None, delimiter=',', dtype=float)
df = df.drop([2,3,4,5,6,8,9,10,11,12,13], axis=1)
df = df[df[0] != 3.0]
df[0] = df[0].replace({1.0: 1, 2.0: -1})
df[0] = df[0].astype(float)
X = df.iloc[:, 1:3].values  # Selects columns 2 and 3
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X = scaler.fit_transform(X) # THIS REALLY HELPED
Y = df.iloc[:, 0].values    # Selects column 1
Y = Y.reshape(-1, 1).astype(float)  
X_and_Y = np.hstack((X, Y))     # Stack them together for shuffling.
np.random.seed(1)               # Set the random seed.
np.random.shuffle(X_and_Y)      # Shuffle the data points in X_and_Y array

print(X.shape)
print(Y.shape)
print(X_and_Y[0])

(130, 2)
(130, 1)
[-0.81784764 -0.61230964 -1.        ]


# 20 training

In [3]:
# Divide the data points into training, validation, and test sets
X_shuffled = X_and_Y[:, :2]  # First two columns as features
Y_shuffled = X_and_Y[:, 2]   # Third column as target

# Compute split indices
train_split = int(0.2 * len(X_shuffled))  # 20% for training

# Create the splits
X_train = X_shuffled[:train_split]
Y_train = Y_shuffled[:train_split]
X_test = X_shuffled[train_split:]
Y_test = Y_shuffled[train_split:]

# Print shapes
print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)


(26, 2)
(26,)
(104, 2)
(104,)


In [4]:
# Perform grid search for best max depth.

# 1. Create a decision tree classifier.
#    Hint: You can use tree.DecisionTreeClassifier()
#          We use "entropy" as the criterion. The random state should be
#          set to 1 for consistent results. Other options are left at default.
estimator = tree.DecisionTreeClassifier(criterion="entropy", random_state=1)
# 2. Create a grid searcher with cross-validation.
D_list = [1, 2, 3, 4, 5]
param_grid = {'max_depth': D_list}
#    Hint: You can use GridSearchCV()
#          Please set a 10-fold cross-validation.
grid_search = GridSearchCV(estimator=estimator, param_grid=param_grid, cv=10)
# 3. Use the grid searcher to fit the training set.
#    - This grid searcher will try every max depth in the list.
#    - For each max depth, a cross-validation is applied to the training set,
#      that is, it creates several (training subset, validation subset) pairs.
#      Note: Sometimes the validation subset is called as "test" subset, but it
#            is not the subset of real test set.
#        - For each pair, a decision tree classifier will be trained on the
#          training subset and evaluated on validation subset.
#        - The average validation scores will be kept.
#
#    Hint: You can simply use .fit() function of the grid searcher.

grid_search.fit(X_train, Y_train)

GridSearchCV(cv=10,
             estimator=DecisionTreeClassifier(criterion='entropy',
                                              random_state=1),
             param_grid={'max_depth': [1, 2, 3, 4, 5]})

In [5]:
cross_val_errors = 1 - grid_search.cv_results_['mean_test_score'].reshape(-1,1)
print("cross score: {}".format(1-cross_val_errors))
best_max_depth = grid_search.best_params_['max_depth']
print("Best max depth D: {}".format(best_max_depth))
test_error = 1 - sum(grid_search.best_estimator_.predict(X_test) == Y_test) / len(X_test)
print("Test error: {}".format(test_error))
train_error = 1 - sum(grid_search.best_estimator_.predict(X_train) == Y_train) / len(X_train)
print("Train error: {}".format(train_error))

cross score: [[0.81666667]
 [0.86666667]
 [0.86666667]
 [0.86666667]
 [0.86666667]]
Best max depth D: 2
Test error: 0.09615384615384615
Train error: 0.038461538461538436


# 50 training

In [6]:
# Divide the data points into training, validation, and test sets
X_shuffled = X_and_Y[:, :2]  # First two columns as features
Y_shuffled = X_and_Y[:, 2]   # Third column as target

# Compute split indices
train_split = int(0.5 * len(X_shuffled))  # 20% for training

# Create the splits
X_train = X_shuffled[:train_split]
Y_train = Y_shuffled[:train_split]
X_test = X_shuffled[train_split:]
Y_test = Y_shuffled[train_split:]

# Print shapes
print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)


(65, 2)
(65,)
(65, 2)
(65,)


In [7]:
# Perform grid search for best max depth.

# 1. Create a decision tree classifier.
#    Hint: You can use tree.DecisionTreeClassifier()
#          We use "entropy" as the criterion. The random state should be
#          set to 1 for consistent results. Other options are left at default.
estimator = tree.DecisionTreeClassifier(criterion="entropy", random_state=1)
# 2. Create a grid searcher with cross-validation.
D_list = [1, 2, 3, 4, 5]
param_grid = {'max_depth': D_list}
#    Hint: You can use GridSearchCV()
#          Please set a 10-fold cross-validation.
grid_search = GridSearchCV(estimator=estimator, param_grid=param_grid, cv=10)
# 3. Use the grid searcher to fit the training set.
#    - This grid searcher will try every max depth in the list.
#    - For each max depth, a cross-validation is applied to the training set,
#      that is, it creates several (training subset, validation subset) pairs.
#      Note: Sometimes the validation subset is called as "test" subset, but it
#            is not the subset of real test set.
#        - For each pair, a decision tree classifier will be trained on the
#          training subset and evaluated on validation subset.
#        - The average validation scores will be kept.
#
#    Hint: You can simply use .fit() function of the grid searcher.

grid_search.fit(X_train, Y_train)

GridSearchCV(cv=10,
             estimator=DecisionTreeClassifier(criterion='entropy',
                                              random_state=1),
             param_grid={'max_depth': [1, 2, 3, 4, 5]})

In [8]:
cross_val_errors = 1 - grid_search.cv_results_['mean_test_score'].reshape(-1,1)
print("cross score: {}".format(1-cross_val_errors))
best_max_depth = grid_search.best_params_['max_depth']
print("Best max depth D: {}".format(best_max_depth))
test_error = 1 - sum(grid_search.best_estimator_.predict(X_test) == Y_test) / len(X_test)
print("Test error: {}".format(test_error))
train_error = 1 - sum(grid_search.best_estimator_.predict(X_train) == Y_train) / len(X_train)
print("Train error: {}".format(train_error))

cross score: [[0.87619048]
 [0.87619048]
 [0.87619048]
 [0.89285714]
 [0.89285714]]
Best max depth D: 4
Test error: 0.07692307692307687
Train error: 0.03076923076923077


# 80 training

In [9]:
# Divide the data points into training, validation, and test sets
X_shuffled = X_and_Y[:, :2]  # First two columns as features
Y_shuffled = X_and_Y[:, 2]   # Third column as target

# Compute split indices
train_split = int(0.5 * len(X_shuffled))  # 20% for training

# Create the splits
X_train = X_shuffled[:train_split]
Y_train = Y_shuffled[:train_split]
X_test = X_shuffled[train_split:]
Y_test = Y_shuffled[train_split:]

# Print shapes
print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)


(65, 2)
(65,)
(65, 2)
(65,)


In [10]:
# Perform grid search for best max depth.

# 1. Create a decision tree classifier.
#    Hint: You can use tree.DecisionTreeClassifier()
#          We use "entropy" as the criterion. The random state should be
#          set to 1 for consistent results. Other options are left at default.
estimator = tree.DecisionTreeClassifier(criterion="entropy", random_state=1)
# 2. Create a grid searcher with cross-validation.
D_list = [1, 2, 3, 4, 5]
param_grid = {'max_depth': D_list}
#    Hint: You can use GridSearchCV()
#          Please set a 10-fold cross-validation.
grid_search = GridSearchCV(estimator=estimator, param_grid=param_grid, cv=10)
# 3. Use the grid searcher to fit the training set.
#    - This grid searcher will try every max depth in the list.
#    - For each max depth, a cross-validation is applied to the training set,
#      that is, it creates several (training subset, validation subset) pairs.
#      Note: Sometimes the validation subset is called as "test" subset, but it
#            is not the subset of real test set.
#        - For each pair, a decision tree classifier will be trained on the
#          training subset and evaluated on validation subset.
#        - The average validation scores will be kept.
#
#    Hint: You can simply use .fit() function of the grid searcher.

grid_search.fit(X_train, Y_train)

GridSearchCV(cv=10,
             estimator=DecisionTreeClassifier(criterion='entropy',
                                              random_state=1),
             param_grid={'max_depth': [1, 2, 3, 4, 5]})

In [11]:
cross_val_errors = 1 - grid_search.cv_results_['mean_test_score'].reshape(-1,1)
print("cross score: {}".format(1-cross_val_errors))
best_max_depth = grid_search.best_params_['max_depth']
print("Best max depth D: {}".format(best_max_depth))
test_error = 1 - sum(grid_search.best_estimator_.predict(X_test) == Y_test) / len(X_test)
print("Test error: {}".format(test_error))
train_error = 1 - sum(grid_search.best_estimator_.predict(X_train) == Y_train) / len(X_train)
print("Train error: {}".format(train_error))

cross score: [[0.87619048]
 [0.87619048]
 [0.87619048]
 [0.89285714]
 [0.89285714]]
Best max depth D: 4
Test error: 0.07692307692307687
Train error: 0.03076923076923077
